##### Importing Modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

##### Variables Declaration

In [13]:
QUESION_COLS = ['question1', 'question2']
TARGET_COL = ['is_duplicate']

##### Basic EDA

In [6]:
df = pd.read_csv("data_files/train.csv")

print("Shape:", df.shape)
df.head(5)

Shape: (404290, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
# Using only 50000 data points to reduce computation
sample_df = df.sample(50000, random_state=200)
sample_df.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
81194,81194,137904,46840,Is there any popular service similar to Quora?,What are some similar websites to Quora?,1
181271,181271,277721,277722,Whatever happened to Kurt Thomas?,What do future Chinese athletes study at sport...,0
32565,32565,1454,59926,Why is Saltwater taffy candy imported in Laos?,Why is Saltwater Taffy candy imported in Mexico?,1
29667,29667,8829,19732,What is the best joke you've ever heard? Pleas...,Tell a joke that the funniest you have heard?,1
271673,271673,389809,389810,Information systems are too important to be le...,Information systems are too important to be le...,1


In [12]:
print("NAN in sample_df: \n", sample_df.isnull().sum())
print("\nDuplicate rows:", sample_df.duplicated().sum())

NAN in sample_df: 
 id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

Duplicate rows: 0


##### Data Encoding
Using top 3000 features (words) from entire corpus, encoding all 100000 questions and then concatenating question1_vectors and question2_vectors horizentally, making a format like it was before.

In [43]:
count_vectorizer = CountVectorizer(max_features=3000)
question1_colvals = sample_df[QUESION_COLS[0]]
question2_colvals = sample_df[QUESION_COLS[1]]
all_questions = list(question1_colvals) + list(question2_colvals)

bow_feats_all_questions = count_vectorizer.fit_transform(all_questions).toarray()

question1_colvals_encoded, question2_colvals_encoded = np.vsplit(bow_feats_all_questions, 2)

# stacking question1 and question2 encoded vectors horizentally to get the format similar to original df
question1_colvals_encoded = pd.DataFrame(question1_colvals_encoded)
question2_colvals_encoded = pd.DataFrame(question2_colvals_encoded)
encoded_sample_df = pd.concat([question1_colvals_encoded, question2_colvals_encoded], axis=1)

del question1_colvals, question2_colvals, all_questions, question1_colvals_encoded, question2_colvals_encoded
encoded_sample_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
encoded_sample_df.shape

(50000, 6000)

##### Model Building

In [46]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(encoded_sample_df, sample_df[TARGET_COL], test_size=0.2, random_state=20)

In [50]:
y_test.shape

(10000, 1)

In [52]:
models = [LogisticRegression(), RandomForestClassifier(), GaussianNB()]
results = {}

for model in models:
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    accuracy = accuracy_score(y_test, predictions)

    results[str(model)] = accuracy


c:\Users\Dmm\anaconda3\envs\data-science\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmm\anaconda3\envs\data-science\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Dmm\anaconda3\envs\data-science\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape o

In [53]:
results

{'LogisticRegression()': 0.7093,
 'RandomForestClassifier()': 0.7457,
 'GaussianNB()': 0.585}

The dataset had become very large (50000, 60001). So that's why the warnings came.

##### Random Forest performed the best with 74.5 % accuracy.